In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import diff_classifier.aws as aws
import diff_classifier.utils as ut
import diff_classifier.msd as msd
import diff_classifier.features as ft
import diff_classifier.imagej as ij
import diff_classifier.heatmaps as hm

from scipy.spatial import Voronoi
import scipy.stats as stats
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.cm as cm
import os
import os.path as op
import numpy as np
import numpy.ma as ma
import pandas as pd
import boto3
import itertools

In [2]:
radius = [4.5, 6.0, 7.0]
do_median_filtering = [True, False]
quality = [1.5, 4.5, 8.5]
linking_max_distance = [6.0, 10.0, 15.0]
gap_closing_max_distance = [6.0, 10.0, 15.0]
max_frame_gap = [1, 2, 5]
track_displacement = [0.0, 10.0, 20.0]

sweep = [radius, do_median_filtering, quality, linking_max_distance, gap_closing_max_distance, max_frame_gap,
         track_displacement]
all_params = list(itertools.product(*sweep))

In [ ]:
all_params

In [3]:
local_folder = '.'
prefix = "P1_S1_R_0001_2_2"
name = "{}.tif".format(prefix)
local_im = op.join(local_folder, name)

outputs = np.zeros((len(all_params), len(all_params[0])+2))

In [ ]:
counter = 50
for i in range(50, len(all_params)):
    params = all_params[i]
    outfile = 'Traj_{}_{}.csv'.format(name.split('.')[0], counter)
    msd_file = 'msd_{}_{}.csv'.format(name.split('.')[0], counter)
    iter_name = "{}_{}".format(prefix, counter)

    ij.track(local_im, outfile, template=None, fiji_bin=None, radius=params[0], threshold=0., 
             do_median_filtering=params[1], quality=params[2], x=511, y=511, ylo=1, median_intensity=300.0, snr=0.0, 
             linking_max_distance=params[3], gap_closing_max_distance=params[4], max_frame_gap=params[5],
             track_displacement=params[6])

    traj = ut.csv_to_pd(outfile)
    msds = msd.all_msds2(traj, frames=651)
    msds.to_csv(msd_file)
    gmean1, gSEM1 = hm.plot_individual_msds(iter_name, alpha=0.05)
    print('Successful parameter calculations for {}'.format(iter_name))
    
    outputs[counter, 0:len(params)] = params
    outputs[counter, len(params)] = gmean1[100]
    outputs[counter, len(params)+1] = max(msds['Track_ID'])
    np.savetxt("sensitivity_sweep.csv", outputs, delimiter=",")
    counter = counter+1
    plt.close('all')

/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmpe6s3qwj9.py
Successful parameter calculations for P1_S1_R_0001_2_2_50
/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmpwygohlt0.py
Successful parameter calculations for P1_S1_R_0001_2_2_51
/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmpt_cccspc.py
Successful parameter calculations for P1_S1_R_0001_2_2_52
/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmpaxw3lokj.py
Successful parameter calculations for P1_S1_R_0001_2_2_53
/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmpim7lsn62.py
Successful parameter calculations for P1_S1_R_0001_2_2_54
/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmphby90k37.py
Successful parameter calculations for P1_S1_R_0001_2_2_55
/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmphwhr1kxb.py
Successful parameter calculations for P1_S1_R_0001_2_2_56
/home/ubuntu/Fiji.app/ImageJ-linux

In [ ]:
numpy.savetxt("sensitivity_sweep.csv", outputs, delimiter=",")